<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex08/Pedro_Luis_Azevedo_Costsa/Aula_8_Exerc%C3%ADcio_175857.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nome = "Pedro Luís Azevedo Costa"
print(f'Meu nome é {nome}')

Meu nome é Pedro Luís Azevedo Costa


#  Exercício: Modelo de Linguagem com auto-atenção

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Este exercício é similar ao da Aula 7, mas iremos agora treinar uma rede neural *com auto-atenção* para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)



O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [3]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Importação dos pacotes

In [4]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [5]:
# Check which GPU we are using
!nvidia-smi

Wed May 25 22:52:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [7]:
from typing import List


def tokenize(text: str, tokenizer):
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, context_size: int):
        self.examples = []
        for text in tqdm_notebook(texts):
            token_ids = tokenize(text=text, tokenizer=tokenizer)
            if len(token_ids) < context_size + 1:
                continue
            # Compute n-grams:
            for i in range(len(token_ids) - context_size):
                input_ids = token_ids[i:i + context_size]
                target_id = token_ids[i + context_size]
                self.examples.append((input_ids, target_id)) 

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        input_ids, target_id = self.examples[idx]
        return torch.LongTensor(input_ids), target_id

## Testando se a implementação do MyDataset está correta

In [8]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, context_size=3)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 5
print('passou no assert de tamanho do dataset')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[ 3396, 10303,   125],
     [ 1660,  5971,   785],
     [ 5971,   785,   125],
     [  785,   125,  1847],
     [  125,  1847, 13779]])

correct_first_batch_target = torch.LongTensor([13239,   125,  1847, 13779, 15616])

assert torch.equal(first_batch_input, correct_first_batch_input)
print('Passou no assert de input')
assert torch.equal(first_batch_target, correct_first_batch_target)
print('Passou no assert de target')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/2 [00:00<?, ?it/s]

passou no assert de tamanho do dataset
Passou no assert de input
Passou no assert de target


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [9]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt

File ‘sample_brwac.txt’ already there; not retrieving.



In [10]:
# Load datasets
context_size = 9

valid_examples = 100
test_examples = 100
texts = open('sample_brwac.txt').readlines()

print('Truncating for debugging purposes.')
#texts = texts[:500]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, context_size=context_size)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, context_size=context_size)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, context_size=context_size)

Truncating for debugging purposes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/24800 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [11]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 27675945
valid examples: 82070
test examples: 166726


In [12]:
from torch.nn import Embedding, Parameter, Linear, Softmax, ReLU
from torch import matmul, transpose
from torch.nn.functional import softmax


#https://www.youtube.com/watch?v=U0s0f995w14

class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size, context_size, embedding_dim):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            context_size (int): Size of the sequence to consider as context for prediction.
            embedding_dim (int): Dimension of the embedding layer for each word in the context.
        """
        # Escreva seu código aqui.
        super().__init__()
        self.vocab_size = vocab_size
        self.context_size = context_size
        self.embedding_dim = embedding_dim


        self.embedding = Embedding(vocab_size, embedding_dim)

        self.K_weight = Linear(self.embedding_dim, self.embedding_dim, bias=False)
        self.Q_weight = Linear(self.embedding_dim, self.embedding_dim, bias=False)
        self.V_weight = Linear(self.embedding_dim, self.embedding_dim, bias=False)
        self.E_weight = Linear(self.embedding_dim, embedding_dim, bias=False)

        context_emb = context_size * embedding_dim
        #print(context_size, embedding_dim)

        #self.linear1 = Linear(embedding_dim, context_emb * 4)
        #self.linear2 = Linear(context_emb *  4, context_emb * 16)
        #self.linear3 = Linear(context_emb * 16 , vocab_size)

        #self.single_linear = Linear(embedding_dim, vocab_size)
        self.linear1 = Linear(embedding_dim, embedding_dim)
        self.linear2 = Linear(embedding_dim, 2*embedding_dim)
        self.linear3 = Linear(2*embedding_dim, vocab_size)


        self.relu = ReLU()





    def forward(self, inputs):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, context_size)
            
        Returns:
            logits of shape (batch_size, vocab_size)
        """

        emb = self.embedding(inputs)
        #emb = torch.rand(50 , 9, 64)

        K = emb.to(device)
        Q = emb[:, -1, :].unsqueeze(1).to(device)
        V = emb.to(device)

        K = self.K_weight(K)
        Q = self.Q_weight(Q)
        V = self.V_weight(V)

        scores = matmul(Q, K.transpose(2,1))
        probs = softmax(scores, dim=-1)
        E = matmul(probs, V)
        E = self.E_weight(E)
        #print(E.shape)

        #print(E.view(inputs.shape[0], -1).shape)

        # logits = self.linear1(E.view(-1, E.shape[1] * E.shape[2]))
        # logits = self.relu(logits)
        # logits = self.linear2(logits)
        # logits = self.linear3(logits)

        # logits = self.single_linear(E.view(-1, E.shape[1] * E.shape[2])) 
        # logits = self.relu(logits)
        logits = self.linear1(E.view(-1, E.shape[1] * E.shape[2])) 
        logits = self.relu(logits)
        logits = self.linear2(logits)
        logits = self.linear3(logits)
        
        return logits




## Teste o modelo com um exemplo

In [13]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
).to(device)

sample_train, _ = next(iter(DataLoader(training_dataset)))
sample_train_gpu = sample_train.to(device)
model(sample_train_gpu).shape

torch.Size([1, 29794])

In [14]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 11585762


## Assert da Perplexidade


In [15]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, vocab_size)
        target: a LongTensor of shape (batch_size,)

    Returns:
        A float corresponding to the perplexity
    """
    print(logits.shape, target.shape)
    loss = nn.functional.cross_entropy(logits, target, reduction='mean')
    return torch.exp(loss)


n_examples = 1000

sample_train, target_token_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
sample_train_gpu = sample_train.to(device)
target_token_ids = target_token_ids.to(device)
logits = model(sample_train_gpu)

my_perplexity = perplexity(logits=logits, target=target_token_ids)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

torch.Size([1000, 29794]) torch.Size([1000])
my perplexity:              29968
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [18]:




max_examples = 100_000_000
eval_every_steps = 10000
lr = 3e-4


# model = LanguageModel(
#     vocab_size=tokenizer.vocab_size,
#     context_size=context_size,
#     embedding_dim=128,
# ).to(device)

train_loader = DataLoader(training_dataset, batch_size=128, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=128)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


try:
    PATH = "langugagemodelv2.pt"
    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    n_examples = checkpoint['n_examples']
except:
    epoch = 0
    loss = 0
    n_examples=0





def train_step(input, target):
    model.train()
    model.zero_grad()

    logits = model(input.to(device))
    loss = nn.functional.cross_entropy(logits, target.to(device))
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()


train_losses = []
#n_examples = 0
step = epoch
while n_examples < max_examples:
    for input, target in train_loader:
        loss = train_step(input.to(device), target.to(device)) 
        train_losses.append(loss)

        #print(step)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            EPOCH = step
            PATH = "langugagemodelv2.pt"
            LOSS = loss

            torch.save({
                        'epoch': step,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'loss': LOSS,
                        'n_examples':n_examples
                        }, PATH)

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(input.to(device), target.to(device))
                    for input, target in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(input)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

620000 steps; 0 examples so far; train ppl: 428.13, valid ppl: 277.25
630000 steps; 1280000 examples so far; train ppl: 269.43, valid ppl: 277.83
640000 steps; 2560000 examples so far; train ppl: 270.72, valid ppl: 277.08
650000 steps; 3840000 examples so far; train ppl: 270.80, valid ppl: 278.60
660000 steps; 5120000 examples so far; train ppl: 272.30, valid ppl: 278.03
670000 steps; 6400000 examples so far; train ppl: 272.29, valid ppl: 278.51
680000 steps; 7680000 examples so far; train ppl: 272.74, valid ppl: 275.89
690000 steps; 8960000 examples so far; train ppl: 272.01, valid ppl: 275.84
700000 steps; 10240000 examples so far; train ppl: 273.11, valid ppl: 276.64
710000 steps; 11520000 examples so far; train ppl: 273.63, valid ppl: 275.43
720000 steps; 12800000 examples so far; train ppl: 272.43, valid ppl: 276.49
730000 steps; 14080000 examples so far; train ppl: 272.56, valid ppl: 274.70
740000 steps; 15360000 examples so far; train ppl: 273.41, valid ppl: 274.87
750000 steps;

KeyboardInterrupt: ignored

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [19]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 257.72151383159144


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [20]:
prompt = 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de comer pizza pois me faz com
Eu gosto de comer pizza pois me faz com a
Eu gosto de comer pizza pois me faz com a sua
Eu gosto de comer pizza pois me faz com a sua vida
Eu gosto de comer pizza pois me faz com a sua vida,
Eu gosto de comer pizza pois me faz com a sua vida, o
Eu gosto de comer pizza pois me faz com a sua vida, o que
Eu gosto de comer pizza pois me faz com a sua vida, o que o
Eu gosto de comer pizza pois me faz com a sua vida, o que o que
Eu gosto de comer pizza pois me faz com a sua vida, o que o que o
Eu gosto de comer pizza pois me faz com a sua vida, o que o que o que
Eu gosto de comer pizza pois me faz com a sua vida, o que o que o que o
Eu gosto de comer pizza pois me faz com a sua vida, o que o que o que o que
Eu gosto de comer pizza pois me faz com a sua vida, o que o que o que o que o
Eu gosto de comer pizza pois me faz com a sua vida, o que o que o que o que o que
Eu gosto de comer pizza pois me faz com a sua vida, o que o que o que o que o que o
Eu go

In [41]:
prompt = 'As árvores da floresta Amazônica são  '
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

As árvores da floresta Amazônica são os
As árvores da floresta Amazônica são os mais
As árvores da floresta Amazônica são os mais de
As árvores da floresta Amazônica são os mais de um
As árvores da floresta Amazônica são os mais de um dos
As árvores da floresta Amazônica são os mais de um dos Santos
As árvores da floresta Amazônica são os mais de um dos Santos,
As árvores da floresta Amazônica são os mais de um dos Santos, o
As árvores da floresta Amazônica são os mais de um dos Santos, o que
As árvores da floresta Amazônica são os mais de um dos Santos, o que o
As árvores da floresta Amazônica são os mais de um dos Santos, o que o que
As árvores da floresta Amazônica são os mais de um dos Santos, o que o que o
As árvores da floresta Amazônica são os mais de um dos Santos, o que o que o que
As árvores da floresta Amazônica são os mais de um dos Santos, o que o que o que o
As árvores da floresta Amazônica são os mais de um dos Santos, o que o que o que o que
As árvores da floresta Amazô